### Defining the loss function.

We want our loss function to combine two aspects: firing correctness (HH firing event) and energy consumption.

1. Firing term: $F(t)$ to be observed firing rate and $F_{targ}(t)$ to be target firing time at t s.t. $T$ is total time of simulation. We can measure our "correctness" as $L_{fire} = \frac{1}{T}\int_0^T (F(t)-F_{targ}(t))^2 dt$. 

2. Energy Consumption: Encourages waveform to be as energy efficient as possible. Suppose we simplify our problem ot step current input, then we are looking at duration and amplitude of the step.Suppose we have some input described as I, $L_{energy} = I_{max} \times I_{dur}$. 

Combining these terms, we can create our lagrangian. 

$$\mathcal{L} = \alpha L_{fire} + \beta I_{dur}$$

s.t. $\alpha, \beta$ are weighting coefficients 